# Neural Belief Propagation Auto-Encoder - AE (64,18) Study
The proposed architecture is configured to learned (64,18) codes and compared with conventional LDPC codes designed using the PEG method.

In [ ]:
# sanity check
from platform import python_version
print(f'python: {python_version()}')

import tensorflow as tf
print(f'tensorflow: {tf.version.VERSION}')

physical_devices_available = tf.config.list_physical_devices()
print(f'Available physical devices: {physical_devices_available}')

import os
os.environ['TF_GPU_THREAD_MODE']='gpu_private'

In [ ]:
import os
import numpy as np

# Get the code G and H matrices
(n,k)=(64,18)
codename = f'LDPC/n{n}_k{k}_m46_dv2-4_dc3-4'
#! codename = f'hamming_{n}_{k}'
code_path = os.path.join('./','encoders/linearblockencoders_reference/', f'{codename}.npz')

code_file = np.load(code_path)
# print(code_file.files)


G_sys = tf.convert_to_tensor(code_file['G_systematic'], dtype=tf.float32)
G_nsys = tf.convert_to_tensor(code_file['G_non_systematic'], dtype=tf.float32)
H_sys = tf.convert_to_tensor(code_file['H_systematic'], dtype=tf.float32)
H_nsys = tf.convert_to_tensor(code_file['H_non_systematic'], dtype=tf.float32)

tf.print(G_sys,summarize=-1)
tf.print(G_nsys,summarize=-1)
tf.print(H_sys,summarize=-1)
tf.print(H_nsys,summarize=-1)
tf.print(tf.matmul(G_sys,tf.transpose(H_sys))%2,summarize=-1)
tf.print(tf.matmul(G_sys,tf.transpose(H_nsys))%2,summarize=-1)
tf.print(tf.matmul(G_nsys,tf.transpose(H_sys))%2,summarize=-1)
tf.print(tf.matmul(G_nsys,tf.transpose(H_nsys))%2,summarize=-1)


In [ ]:
from dataset import random_messages_dataset, random_messages_base_all_zero_all_one_dataset #!, random_messages_base_dataset, all_zero_dataset,
"""
#Dataset logic:
.take() allow to generate a fixed number of batch of the dataset
.cache() allow to use cached data (i.e doesn't execute previous dataset generation instructions)
.shuffle() suffle elements inside batches by picking randomly elmts in a buffer of size buffer_size. similarly to batching procedure, shuffling method of model.fit isn't called when using dataset. it's important to manualy specify a shuffling strategy then.
.prefetch() is used to overlap the processing time of the data producer and data consumer. should be used as last step. 
Here the step before .cache() should be done once and for all at first call of the train_dataset object. The operations after .cache() should be called at each time the dataset is called (ie for each batch in the training).
All the steps before .prefetch() are supposed to be prepared during the training step of the data consumer to be ready when the consumer needs more data.
"""

#TRAIN
epochs=1000
steps_per_epoch=25
train_batch = 64
train_seed = 42
#train_dataset = random_messages_dataset(k, batch=64, prefetch=tf.data.AUTOTUNE, seed=train_seed)
#train_dataset = random_messages_base_dataset(k, batch=64, prefetch=tf.data.AUTOTUNE, seed=train_seed)
#train_dataset = all_zero_dataset(k, batch=64, prefetch=tf.data.AUTOTUNE)
#train_dataset = random_messages_base_all_zero_all_one_dataset(k, batch=train_batch, prefetch=tf.data.AUTOTUNE, seed=train_seed)#.take(steps_per_epoch*epochs).cache().prefetch(tf.data.AUTOTUNE)#.shuffle(buffer_size=train_batch*steps_per_epoch)
train_dataset = random_messages_base_all_zero_all_one_dataset(k, batch=train_batch, prefetch=tf.data.AUTOTUNE, seed=train_seed).take(steps_per_epoch * epochs).cache()

#VALIDATION
validation_seed = 43
val_batch_size = 64
validation_dataset = random_messages_dataset(k, batch=val_batch_size, prefetch=tf.data.AUTOTUNE, seed=validation_seed).take(int(8000 / val_batch_size)).cache()

#TEST
#!N_Bytes = 20000 * k
#!N_bits = N_Bytes * 8
#!N_words = N_bits / k
test_batch_size = 100  #!
test_seed = 44
test_dataset = random_messages_dataset(
        k,
        batch=test_batch_size,  #!
        prefetch=tf.data.AUTOTUNE,
        seed=test_seed,  #!
    )#!.take(int(N_words / test_batch_size)).cache()


In [ ]:
from tools import ebno_db_to_snr_db
# training and evaluation setup
# 
#ebn0_training_dbs = tf.range(0, 9, delta=1, dtype=tf.float32)
ebn0_training_dbs = 4
ebn0_eval_dbs = tf.range(0, 7, delta=1, dtype=tf.float32)
noise_power_training_dbs = -ebno_db_to_snr_db(ebn0_training_dbs, k/n)



In [ ]:
from tools import create_paths_and_summaries

paths_and_summaries = create_paths_and_summaries('study-ae-paper\\study-ae-{n}-{k}'.format(n=n,k=k), 'Eb/N0 (dB)', ebn0_eval_dbs)
summary_ber = paths_and_summaries.summary_ber
summary_bler = paths_and_summaries.summary_bler
summary_bec = paths_and_summaries.summary_bec
summary_blec = paths_and_summaries.summary_blec
summary_bpci_ber = paths_and_summaries.summary_bpci_ber
summary_bpci_bler = paths_and_summaries.summary_bpci_bler
models_path = paths_and_summaries.models_path
tensorboard_path = paths_and_summaries.tensorboard_path

In [ ]:
# model definition
from autoencoders import AutoEncoder
from metrics import BitErrorRate, BlockErrorRate, BinomialProportionConfidenceInterval, BitErrorCount, BlockErrorCount

noise_power_training_dbs = -ebno_db_to_snr_db(ebn0_training_dbs, k/n)

def create_model(
    n,
    k,
    build_dataset,  #! Used to build model graph at creation
    conf="A",
    learning_rate=1e-1,
    model_index=None,
    training_noise_power_db=[0.0],
    train_model=True,
    G=None,
    H=None,
    trainable_code=True,
    trainable_decoder=True,
    additional_confs=[],
    name=None,
):
    if name is None:
        conf_string = str(conf) if conf is not None else ""
        lr_string = str(learning_rate) if learning_rate is not None else ""
        index_string = str(model_index) if model_index is not None else ""
        train_model_string = str(train_model) if train_model is not None else ""
        trainable_code_string = (
            str(trainable_code) if trainable_code is not None else ""
        )
        name = f"AE-{n}-{k}-{conf_string}-{index_string}"  #!-{lr_string} float number harmful because of the decimal dot in file name eg 0.01
    else:
        name = name

    n_iter = 5
    model = AutoEncoder(
        n,
        k,
        n_iter,
        conf,
        training_noise_power_db=training_noise_power_db,
        G=G,
        H=H,
        trainable_code=trainable_code,
        trainable_decoder=trainable_decoder,
        name=name,
    )

    metric_list = [
            BitErrorRate(name="BER",from_logits=False),
            BlockErrorRate(name="BLER",from_logits=False),
            BitErrorCount(name="BEC",from_logits=False, mode="sum"), 
            BlockErrorCount(name="BLEC",from_logits=False, mode="sum"), 
            BinomialProportionConfidenceInterval(monitor_class=BitErrorRate, monitor_params={"name":"bpci_ber","from_logits":False},fraction=0.95, name="BPCI_BER"),
            BinomialProportionConfidenceInterval(monitor_class=BlockErrorRate, monitor_params={"name":"bpci_bler","from_logits":False},fraction=0.95, name="BPCI_BLER")
        ]
    
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=metric_list,
    )

    
    
    # Build model graph using build dataset
    build_datum = list(build_dataset.take(1))  #!
    model(build_datum[0][0])  #!
    # model(np.array([train_dataset[0]]))
    # tf.print(model.get_weights())
    return model

In [ ]:
from callbacks.defaults import default_training_callbacks, default_configuration_early_stopping, default_configuration_tensorboard, default_configuration_reduce_lr_on_plateau, default_configuration_model_checkpoint

def train_model(
    model,
    models_path,
    train_dataset,
    validation_dataset,
    tensorboard_path,
    epochs=1000,
    steps_per_epoch=25,
):

    ckpt_path = os.path.join(models_path, "checkpoint\checkpoint.tf")
    configuration_tensorboard = default_configuration_tensorboard(tensorboard_path)
    configuration_earlystopping = default_configuration_early_stopping(
        monitor="loss", patience=200
    )
    configuration_reduce_lr_on_plateau = default_configuration_reduce_lr_on_plateau(
        monitor="val_loss", factor=0.8, patience=50
    )
    configuration_model_checkpoint = default_configuration_model_checkpoint(
        filepath=ckpt_path
    ) 
    callbacks = default_training_callbacks(
        configuration_earlystopping=configuration_earlystopping,
        configuration_tensorboard=configuration_tensorboard,
        configuration_reduce_lr_on_plateau=configuration_reduce_lr_on_plateau,
        configuration_model_checkpoint=configuration_model_checkpoint,
    )

    # if needed
    # from dataset import random_messages
    # validation_messages = random_messages(k, 10_000)
    # bit_error_distribution_callback = BitErrorDistribution(tensorboard_path, eval_dataset=validation_messages, from_logits=False)
    # callbacks.append(bit_error_distribution_callback)

    # latent_space_callback = LatentSpaceDistribution(tensorboard_path, eval_dataset=validation_messages)
    # callbacks.append(latent_space_callback)

    # ds = np.array(list(train_dataset))
    with tf.device("/GPU:0"):
        model.fit(
            x=train_dataset,  # tf.reshape(ds[:,0],shape=(50*64,16)),#
            validation_data=validation_dataset,
            # y=train_dataset,#y=tf.reshape(ds[:,1],shape=(50*64,16)),
            epochs=epochs,
            # batch_size=train_batch,
            steps_per_epoch=steps_per_epoch,
            callbacks=callbacks,
            verbose=1,
        )
    model.load_weights(ckpt_path)

In [ ]:
from callbacks import BatchTerminationCallback
# model evaluation
#import os

def ber_ci_condition(_, logs):
    if 'BPCI_BER' in logs:
        epsilon = 1e-7
        (ci_span, ci_low, ber, ci_high) = logs['BPCI_BER']
        return (ci_span)/(ber+epsilon) < 0.1
    else:
        return False

def bler_ci_condition(_, logs):
    if 'BPCI_BLER' in logs:
        epsilon = 1e-7
        (ci_span, ci_low, bler, ci_high) = logs['BPCI_BLER']
        return (ci_span)/(bler+epsilon) < 0.1
    else:
        return False

def evaluate_model(summary_ber, summary_bler, summary_bec, summary_blec, summary_bpci_ber, summary_bpci_bler, models_path, model, ebn0_eval_dbs, test_dataset, k, n):
    # tf.print(model.get_weights())
    bers = []
    blers = []
    becs = []  
    blecs = []
    bpci_bers = [] 
    bpci_blers = []
    
    # mean(Es) assumed to be 1
    noise_power_eval_dbs = -ebno_db_to_snr_db(ebn0_eval_dbs, k / n)
    snr_eval_dbs = ebno_db_to_snr_db(ebn0_eval_dbs, k / n)

    # add SNR values
    summary_ber["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bler["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bec["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_blec["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bpci_ber["SNR(dB)"] = snr_eval_dbs.numpy()
    summary_bpci_bler["SNR(dB)"] = snr_eval_dbs.numpy()

    for ebn0_eval_db, noise_power_eval_db in zip(ebn0_eval_dbs, noise_power_eval_dbs):
        print(
            f"evaluating {model.name} at Eb/N0 [dB]: {ebn0_eval_db} / N0 [dB]: {noise_power_eval_db}"
        )
        model.channel.noise_power_db=noise_power_eval_db

        termination_callback = BatchTerminationCallback(ber_ci_condition)
        summary = model.evaluate(
            test_dataset,  # validation_dataset,
            steps=25000,
            return_dict=True,
            callbacks=[termination_callback]
        )
        
        (ber_ci_span, ci_min, ber_bpci_metric, ci_max) = summary['BPCI_BER']
        (bler_ci_span, ci_min, bler_bpci_metric, ci_max) = summary['BPCI_BLER']
        
        #print(summary)
        ber = summary["BER"]  
        bler = summary["BLER"] 
        bec = summary["BEC"]  
        blec = summary["BLEC"] 
        bpci_ber = summary["BPCI_BER"]  
        bpci_bler = summary["BPCI_BLER"] 
        bers.append(ber) 
        blers.append(bler)
        becs.append(bec) 
        blecs.append(blec)
        bpci_bers.append(bpci_ber) 
        bpci_blers.append(bpci_bler)
        
        print(f"Eb/N0: {ebn0_eval_db} BER: {ber} 95% CI: {ber_ci_span} - BEC: {bec} - BLER: {ber} 95% CI: {bler_ci_span} - BLEC: {blec}")

    # learned_code
    G, H = model.code_generator(None)

    # store results and model
    summary_ber[model.name] = bers
    summary_bler[model.name] = blers
    summary_bec[model.name] = becs
    summary_blec[model.name] = blecs
    summary_bpci_ber[model.name] = bpci_bers
    summary_bpci_bler[model.name] = bpci_blers

    model_path = os.path.join(models_path, model.name)
    encoder_path = os.path.join(model_path, model.encoder.name)
    decoder_path = os.path.join(model_path, model.decoder.name)
    code_generator_path = os.path.join(model_path, model.code_generator.name)
    matrices_path = os.path.join(model_path, "matrices")

    print(model_path, encoder_path, decoder_path, code_generator_path)
    os.makedirs(encoder_path, exist_ok=True)
    os.makedirs(decoder_path, exist_ok=True)
    os.makedirs(code_generator_path, exist_ok=True)
    os.makedirs(matrices_path, exist_ok=True)
    print(f" saving model {model.name} in {model_path}")
    # tf.keras.utils.plot_model(model, to_file=model_path+'/model.png', show_shapes=True, show_dtype=False,show_layer_names=True, rankdir='TB', expand_nested=True, dpi=96)
    # model.save(model_path,overwrite=True)
    model.encoder.save(encoder_path, overwrite=True)
    model.decoder.save(decoder_path, overwrite=True)
    model.code_generator.save(code_generator_path, overwrite=True)
    np.savetxt(matrices_path + "\G.csv", np.array(G), fmt="%i")
    np.savetxt(matrices_path + "\H.csv", np.array(H), fmt="%i")
    
    #x = list(test_dataset.take(1))[0][0][0:10]
    #tf.print(tf.cast(x,dtype=tf.int32),summarize=-1)
    #tf.print(tf.cast(model(x),dtype=tf.int32),summarize=-1)

In [ ]:

from collections import namedtuple
from tools import configurations_product,configurations_list

#print(tf.executing_eagerly())
#tf.config.run_functions_eagerly(True)
options = ['n','k','build_dataset','conf','learning_rate','model_index','training_noise_power_db','train_model','G','H','trainable_code','trainable_decoder','additional_confs','name']

ML_eval_conf = configurations_list(options,[[n,k,train_dataset,'ML',None,0,noise_power_training_dbs,False,None,None,False,False,[],"ML"]])[0]
BP_eval_conf = configurations_list(options,[[n,k,train_dataset,'BP',None,0,noise_power_training_dbs,False,None,None,False,False,[],"BP"]])[0]
GNBP_eval_conf = configurations_list(options,[[n,k,train_dataset,'GNBP',1e-1,0,noise_power_training_dbs,True,None,None,False,True,[],"GNBP"]])[0]

n_trials= 5

config_list = [[n,k,train_dataset,'A',1e-1,i,noise_power_training_dbs,True,None,None,True,True,[BP_eval_conf],f'AE_GNBP_{i}'] for i in range(n_trials)]                \
+[[n,k,train_dataset,'BP',None,0,noise_power_training_dbs,False,G_sys,H_sys,False,False,[],'LDPC_SYS_BP']]                                                        \
+[[n,k,train_dataset,'GNBP',1e-1,i,noise_power_training_dbs,True,G_sys,H_sys,False,True,[],f'LDPC_SYS_GNBP_{i}'] for i in range(n_trials)]                       \
+[[n,k,train_dataset,'BP',None,0,noise_power_training_dbs,False,G_sys,H_nsys,False,False,[],'LDPC_NSYS_BP']]                                                       \
+[[n,k,train_dataset,'GNBP',1e-1,i,noise_power_training_dbs,True,G_sys,H_nsys,False,True,[],f'LDPC_NSYS_GNBP_{i}'] for i in range(n_trials)]                        

#TODO: improve configuration and default configuration + add n trials as a configuration param
#print(config_list)
configurations = configurations_list(options,config_list)
for c in configurations:
    #print(c)
    model = create_model(**c._asdict())
    if c.train_model:
        train_model(model, models_path, train_dataset, validation_dataset, tensorboard_path,epochs,steps_per_epoch)
    (G,H) = model.code_generator(None)
    tf.assert_equal(tf.math.floormod(tf.matmul(G,tf.transpose(H)),2), tf.zeros(shape=(k,(n-k))),message="Generator and PC matrices are not matched as syndrome matrix (G.H^T) is not equal to 0")
    evaluate_model(summary_ber, summary_bler, summary_bec, summary_blec, summary_bpci_ber, summary_bpci_bler, models_path, model, ebn0_eval_dbs, test_dataset, k, n)
    
    if c.additional_confs != []:
        parent_model_name = model.name
        for conf in c.additional_confs:           
            conf = conf._asdict()
            conf['G'] = G
            conf['H'] = H
            conf['name'] = parent_model_name + "_" +conf['name']
            Configuration = namedtuple("Configuration", options)
            conf = Configuration(**conf)
            print("Additionnal Conf:")
            model = create_model(**conf._asdict())
            if conf.train_model:
                train_model(model, models_path, train_dataset, validation_dataset, tensorboard_path, epochs, steps_per_epoch)
            evaluate_model(summary_ber, summary_bler, summary_bec, summary_blec, summary_bpci_ber, summary_bpci_bler, models_path, model, ebn0_eval_dbs, test_dataset, k, n)




Copyright (c) 2022 Orange

Authors: Guillaume Larue <guillaume.larue@orange.com>, Quentin Lampin <quentin.lampin@orange.com>, Louis-Adrien Dufrene <louisadrien.dufrene@orange.com>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), 
to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice (including the next paragraph) shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS 
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER 
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE